# 🚦 SRIG — TabTransformer Model Training
### Smart Road India Grid | Model 3 — Civic Score Predictor
**Yash Jani | Red and White Institute, Surat**

---

## 🧠 Yeh Model Kaise Kaam Karta Hai?

```
Driver ka Data (10 features)
         ↓
   Input Projection
   (numbers → vectors)
         ↓
  Transformer Encoder
  (patterns seekhta hai)
         ↓
  Classification Head
  (final decision)
         ↓
  Risk Level Output
  (PLATINUM / GOLD / SILVER / BRONZE / HIGH_RISK)
```

### 🔑 Key Concept — Transformer kya hota hai?
Transformer ek **attention-based** model hai. Matlab yeh automatically samajhta hai ki konsa feature zyada important hai.

Example:
- `drunk_driving_cases = 2` → Transformer is pe **bahut zyada attention** dega
- `city_tier = 1` → Transformer is pe **kam attention** dega

Yeh CNN ya simple neural network se zyada smart hota hai! 🚀

## 📦 Step 1 — Libraries Import Karo

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Device check — GPU hai to GPU, nahi to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ Libraries import ho gayi!')
print(f'💻 Device: {device}')
print(f'🔥 PyTorch Version: {torch.__version__}')

## 📂 Step 2 — Dataset Load Karo

Pehle humne jo `civic_score_dataset.csv` banaya tha, use load karenge.

In [ ]:
df = pd.read_csv('data/civic_score_dataset.csv')

print(f'✅ Dataset loaded!')
print(f'   Rows    : {df.shape[0]:,}')
print(f'   Columns : {df.shape[1]}')
print(f'\n📊 Risk Level Distribution:')
print(df['risk_level'].value_counts())
df.head()

## ⚙️ Step 3 — Data Preprocessing

Model ko raw data directly nahi dete — pehle prepare karna padta hai:

1. **Label Encoding** — `PLATINUM`, `GOLD` etc → `0, 1, 2, 3, 4` (numbers mein)
2. **Standard Scaling** — sab features ko same range mein laate hain (0 ke around)
3. **Train/Test Split** — 80% training, 20% testing

In [ ]:
FEATURE_COLS = [
    'helmet_compliance_days', 'signal_violations', 'speed_violations',
    'school_zone_violations', 'drunk_driving_cases', 'accident_history',
    'years_driving', 'training_completed', 'complaints_filed', 'city_tier'
]
TARGET_COL = 'risk_level'

X = df[FEATURE_COLS].values
y = df[TARGET_COL].values

# Label Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)
NUM_CLASSES = len(le.classes_)
print(f'✅ Classes: {le.classes_}')
print(f'   Encoded as: {list(range(NUM_CLASSES))}')

# Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f'\n✅ Scaling done!')
print(f'   Before scaling - Mean: {X[:,0].mean():.1f}, Std: {X[:,0].std():.1f}')
print(f'   After scaling  - Mean: {X_scaled[:,0].mean():.2f}, Std: {X_scaled[:,0].std():.2f}')

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)
print(f'\n✅ Split done!')
print(f'   Train samples : {len(X_train):,}')
print(f'   Test samples  : {len(X_test):,}')

## 🗂️ Step 4 — PyTorch Dataset Class Banao

PyTorch ko data ek specific format mein chahiye — `Dataset` class se batate hain ki data kahan se aayega aur kaise dena hai.

In [ ]:
class CivicScoreDataset(Dataset):
    def __init__(self, X, y):
        # numpy arrays ko PyTorch tensors mein convert karo
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Dataset objects banao
train_dataset = CivicScoreDataset(X_train, y_train)
test_dataset  = CivicScoreDataset(X_test,  y_test)

# DataLoader — batch mein data deta hai model ko
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False)

print(f'✅ Dataset ready!')
print(f'   Batch size     : 64')
print(f'   Train batches  : {len(train_loader)}')
print(f'   Test batches   : {len(test_loader)}')

# Ek sample dekho
sample_X, sample_y = next(iter(train_loader))
print(f'\n📐 Sample batch shape:')
print(f'   X shape : {sample_X.shape}  → (64 drivers, 10 features)')
print(f'   y shape : {sample_y.shape}  → (64 labels)')

## 🤖 Step 5 — TabTransformer Model Define Karo

### Architecture Samjho:

```
Input: [10 features]
         ↓
Input Projection Layer
(10 → 64 dimensions)
         ↓
┌─────────────────────┐
│  Transformer Layer 1 │  ← Self-Attention + Feed Forward
│  Transformer Layer 2 │  ← Self-Attention + Feed Forward  
│  Transformer Layer 3 │  ← Self-Attention + Feed Forward
└─────────────────────┘
         ↓
Layer Normalization
         ↓
Linear(64 → 64) + ReLU
         ↓
Linear(64 → 5 classes)
         ↓
Output: [PLATINUM, GOLD, SILVER, BRONZE, HIGH_RISK]
```

### 🔑 Self-Attention kya karta hai?
Model khud decide karta hai ki training ke dauran **konse features zyada important hain**. Jaise:
- `drunk_driving` → high attention
- `city_tier` → low attention

In [ ]:
class TabTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=3, dropout=0.1):
        super(TabTransformer, self).__init__()

        # Step A: Input ko d_model dimensions mein project karo
        # 10 features → 64 dimensional space
        self.input_projection = nn.Linear(input_dim, d_model)

        # Step B: Transformer Encoder
        # nhead=4 matlab 4 alag alag attention heads hain
        # Har head alag pattern seekhta hai
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Step C: Classification Head — final prediction karta hai
        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.input_projection(x)   # (batch, 64)
        x = x.unsqueeze(1)             # (batch, 1, 64) — sequence format
        x = self.transformer(x)        # (batch, 1, 64) — attention applied
        x = x.squeeze(1)               # (batch, 64)
        return self.classifier(x)      # (batch, 5 classes)

# Model banao
model = TabTransformer(
    input_dim=len(FEATURE_COLS),
    num_classes=NUM_CLASSES
).to(device)

# Model ka summary
total_params = sum(p.numel() for p in model.parameters())
print(f'✅ TabTransformer Model Ready!')
print(f'   Total Parameters : {total_params:,}')
print(f'   Input Features   : {len(FEATURE_COLS)}')
print(f'   Output Classes   : {NUM_CLASSES}')
print(f'   Device           : {device}')
print(f'\n🏗️ Model Architecture:')
print(model)

## 🏋️ Step 6 — Model Train Karo

### Training kaise hoti hai?
```
1. Batch of 64 drivers model mein daalo
2. Model prediction karta hai
3. Loss calculate karo (prediction vs actual)
4. Backpropagation — model apni galti se seekhta hai
5. Weights update karo
6. Repeat → 30 epochs tak
```

**Loss** = model kitna galat hai (kam hona chahiye)

**Accuracy** = model kitna sahi hai (zyada hona chahiye)

In [ ]:
# Loss function aur Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

EPOCHS = 30
train_losses     = []
train_accuracies = []

print('🏋️ Training shuru ho rahi hai...\n')
print(f'{"Epoch":<8} {"Loss":<12} {"Accuracy":<12}')
print('-' * 35)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct    = 0
    total      = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()           # gradients reset karo
        outputs = model(X_batch)        # forward pass
        loss = criterion(outputs, y_batch)  # loss calculate karo
        loss.backward()                 # backpropagation
        optimizer.step()                # weights update karo

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(y_batch).sum().item()
        total   += y_batch.size(0)

    scheduler.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)

    if (epoch + 1) % 5 == 0:
        print(f'{epoch+1:<8} {avg_loss:<12.4f} {accuracy:<12.2f}%')

print('\n✅ Training complete!')

## 📈 Step 7 — Training Progress Visualize Karo

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('SRIG TabTransformer — Training Progress', fontsize=14, fontweight='bold')

# Loss Curve
axes[0].plot(range(1, EPOCHS+1), train_losses, color='#E74C3C', linewidth=2.5, marker='o', markersize=3)
axes[0].set_title('Training Loss (Kam hona chahiye ✅)', fontsize=12)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].fill_between(range(1, EPOCHS+1), train_losses, alpha=0.1, color='red')
axes[0].grid(True, alpha=0.3)

# Accuracy Curve
axes[1].plot(range(1, EPOCHS+1), train_accuracies, color='#27AE60', linewidth=2.5, marker='o', markersize=3)
axes[1].set_title('Training Accuracy (Zyada hona chahiye ✅)', fontsize=12)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].fill_between(range(1, EPOCHS+1), train_accuracies, alpha=0.1, color='green')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
os.makedirs('models', exist_ok=True)
plt.savefig('models/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Training curves save ho gayi: models/training_curves.png')

## 🧪 Step 8 — Model Evaluate Karo (Test Data pe)

Training ke baad **test data** pe check karte hain ki model ne actual unseen data pe kitna achha perform kiya.

In [ ]:
model.eval()
all_preds  = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch   = X_batch.to(device)
        outputs   = model(X_batch)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f'🎯 Test Accuracy: {test_accuracy:.2f}%')
print(f'\n📋 Detailed Classification Report:')
print(classification_report(all_labels, all_preds, target_names=le.classes_))

## 🎨 Step 9 — Confusion Matrix Dekho

Confusion matrix se pata chalta hai ki model kahan galti kar raha hai — kaun si class ko kaun si class samajh raha hai.

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=le.classes_,
    yticklabels=le.classes_
)
plt.title('SRIG — Confusion Matrix\n(Diagonal pe zyada numbers = achha model)', fontsize=13, fontweight='bold')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('models/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Confusion matrix save ho gayi: models/confusion_matrix.png')
print('\n💡 Diagonal values zyada hain = model sahi predict kar raha hai!')

## 💾 Step 10 — Model Save Karo

In [ ]:
torch.save({
    'model_state':  model.state_dict(),
    'label_encoder': le,
    'scaler':        scaler,
    'feature_cols':  FEATURE_COLS,
    'classes':       le.classes_.tolist()
}, 'models/civic_score_model.pth')

print('🎉 Model successfully save ho gaya!')
print('   📁 Location : models/civic_score_model.pth')
print(f'   🎯 Accuracy  : {test_accuracy:.2f}%')
print('\n✅ Ab predict.ipynb run karo aur real predictions dekho!')